In [3]:
import gurobipy as gp
from gurobipy import GRB

import pandas as pd
import math

import time
from collections import defaultdict

## Actual Data

In [4]:
banker = pd.read_excel('.//banker_data//banker_data.xlsx')
bank = pd.read_csv('.//bank_loan_data//bank.csv')
loan = pd.read_csv('.//bank_loan_data//loan.csv')
loan_type = loan['loan_type'].unique()   # loan_type = [0,1] 
# type = ["living", "investment"]

# banker['salary (in k)'] = [i*1000 for i in banker['salary (in k)']]
# banker['capacity'] = [i*10000 for i in banker['capacity']]

# P = range(30) #range(round(len(banker)*0.1))
# B = range(2) #range(round(len(bank)*0.1))
# L = range(50) #range(round(len(loan)*0.02))

# T = range(len(loan_type))
# print([P,B,L,T])
        
# # yoinked from ass1 
# dist = [ [0]*len(B) for i in P]
# for p in P:
#      for b in B:
#         # LA,LONG for banker is column 1,2; for banks LONG,LA is 1:2
#         dist[p][b] = math.dist(banker.iloc[p, 1:3],bank.iloc[b, 2:0:-1])


# print("bank\n", bank.head())
# print("loan\n",loan.head())
# print("banker\n",banker.head())

## Dummy Data

In [38]:
P = range(6)
B = range(3)
L = range(15)
T = range(len(loan_type))
print([P,B,L,T])

banker = banker.iloc[0:6]
bank = bank.iloc[0:3]
loan = loan.iloc[0:15]

banker.loc[0, "capacity"] = 750
banker.loc[1,"capacity"] = 250
banker.loc[2, "capacity"] = 500
banker.loc[3,"capacity"] = 250
banker.loc[4, "capacity"] = 750
banker.loc[5,"capacity"] = 1000
# banker.loc[6, "capacity"] = 500
# banker.loc[7,"capacity"] = 250
# banker.loc[8, "capacity"] = 500
# banker.loc[9,"capacity"] = 500

bank.loc[0,"capacity"] = 1000
bank.loc[1,"capacity"] = 5000
bank.loc[2,"capacity"] = 3500

loan.loc[0,"loan_amount"] = 300
loan.loc[0,"loan_type"] = 1
loan.loc[1,"loan_amount"] = 200
loan.loc[1,"loan_type"] = 0
loan.loc[2,"loan_amount"] = 400
loan.loc[2,"loan_type"] = 0
loan.loc[3,"loan_amount"] = 100
loan.loc[3,"loan_type"] = 0
loan.loc[4,"loan_amount"] = 300
loan.loc[4,"loan_type"] = 1
loan.loc[5,"loan_amount"] = 200
loan.loc[5,"loan_type"] = 0
loan.loc[6,"loan_amount"] = 400
loan.loc[6,"loan_type"] = 0
loan.loc[7,"loan_amount"] = 100
loan.loc[7,"loan_type"] = 0
loan.loc[8,"loan_amount"] = 400
loan.loc[8,"loan_type"] = 0
loan.loc[9,"loan_amount"] = 100
loan.loc[9,"loan_type"] = 0
loan.loc[10,"loan_amount"] = 150
loan.loc[10,"loan_type"] = 1
loan.loc[11,"loan_amount"] = 200
loan.loc[11,"loan_type"] = 0
loan.loc[12,"loan_amount"] = 600
loan.loc[12,"loan_type"] = 0
loan.loc[13,"loan_amount"] = 100
loan.loc[13,"loan_type"] = 1
loan.loc[14,"loan_amount"] = 300
loan.loc[14,"loan_type"] = 0
# loan.loc[15,"loan_amount"] = 200
# loan.loc[15,"loan_type"] = 0
# loan.loc[16,"loan_amount"] = 400
# loan.loc[16,"loan_type"] = 0
# loan.loc[17,"loan_amount"] = 100
# loan.loc[17,"loan_type"] = 0
# loan.loc[18,"loan_amount"] = 400
# loan.loc[18,"loan_type"] = 0
# loan.loc[19,"loan_amount"] = 200
# loan.loc[19,"loan_type"] = 1

        
# Dummy distance between banker and branch
dist = [[5,8,1], [6,9,2], [1,2,4], [1,1,1], [4,3,5], [6,2,4]]#, [7,1,11], [8,2,3], [6,2,1], [2,3,2]]

print("bank\n", bank.head())
print("loan\n",loan.head())
print("banker\n",banker.head())

[range(0, 6), range(0, 3), range(0, 15), range(0, 2)]
bank
                                             location        long        lat   
0  Westfield Fountain Gate, 352 Princes Highway, ...  145.304441 -38.018206  \
1               330 Collins St, Melbourne, VIC, 3000  144.963217 -37.816309   
2               460 Collins St, Melbourne, VIC, 3000  144.958852 -37.817673   

   capacity  
0    1000.0  
1    5000.0  
2    3500.0  
loan
    loan_amount  loan_type
0        300.0          1
1        200.0          0
2        400.0          0
3        100.0          0
4        300.0          1
banker
    banker_id   Latitude   Longitude  tenure (in years)  capacity  v1_rand_0_1   
0          1 -37.695528  143.951218           8.183687       750     0.863803  \
1          2 -37.551700  144.405921           8.184793       250     0.539807   
2          3 -37.537275  144.841580           6.404775       500     0.934031   
3          4 -37.563307  144.933624           4.616158       250     0.88

In [39]:
assigned_loans = []
assigned_bankers = []
targets = []
branch = []
branch_dict = defaultdict()

In [40]:
for target in [i/100 for i in range(1, 41)]:
    start_time = time.time()
    m = gp.Model()
    m.Params.LogToConsole = 0
#     m.Params.OutputFlag = 1

    M = 100000
    x = m.addVars(T,L,B, name="x", vtype = GRB.BINARY)
    y = m.addVars(P,B, name="y", vtype = GRB.BINARY)
    z = m.addVars(T,L,P, name="z", vtype = GRB.BINARY)

    # Objective
    m.setObjective(gp.quicksum( dist[p][b]*y[p,b] for b in B for p in P) +
                    gp.quicksum( banker.iloc[p]["salary (in k)"]*y[p,b] for p in P for b in B),
                   GRB.MINIMIZE )

    ## Constraints ##
    # Loans # 
    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B)
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                 name='1.each loan <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] for p in P) 
                   <= 1.5 for t in T for l in L if loan.iloc[l]["loan_type"]==t),
                 name='2.each loan to <= 1 banker')

    m.addConstrs( (gp.quicksum( y[p,b] for b in B) 
                   <= 1.5 for p in P),
                 name='3.each banker <= 1 bank')

    m.addConstrs( (gp.quicksum( z[t,l,p] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <=  banker.iloc[p]["capacity"] for p in P ),
                 name='4.value of loans respect banker cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   <= bank.iloc[b]["capacity"] for b in B ), 
                 name='5.total loan respect bank cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] * loan.iloc[l]["loan_amount"] 
                               for t in T for l in L if loan.iloc[l]["loan_type"]==t) 
                   >= target * bank.iloc[b]["capacity"] for b in B),
                 name='6.each bank spend 10% of cap')

    m.addConstrs( (gp.quicksum( x[t,l,b] for b in B ) == gp.quicksum( z[t,l,p] for p in P )
                for t in T for l in L if loan.iloc[l]["loan_type"]==t ),
                name='7. if 1 loan to bank -> 1 loan to a banker')

    m.addConstrs( ( x[t,l,b] + z[t,l,p] <= 1.5 + y[p,b] 
                   for t in T for l in L if loan.iloc[l]["loan_type"]==t for b in B for p in P),
                name='8.Link z and y')


    #########################################

    # Constraint: The value of investment loans cannot exceed 50% of total for each bank
    m.addConstrs((gp.quicksum(loan.loc[l, "loan_amount"] * x[1, l, b] for l in L if loan.iloc[l]["loan_type"]==1) <=
                  (gp.quicksum(0.5 * loan.loc[l, "loan_amount"] * x[t, l, b] for t in T for l in L if loan.iloc[l]["loan_type"]==t))
                 for b in B),
                 name="9.investment loan <= 50% percent per bank")


    #########################################
    print('optimizing...')
    m.optimize()

    for v in m.getVars():
        if v.x > 1e-6:
            if v.varName[0] == 'z':
                assigned_bankers.append(int(v.varName[2:-1].split(',')[2])+1)
                assigned_loans.append(int(v.varName[2:-1].split(',')[1]))
                targets.append(target)
                branch.append(branch_dict[int(v.varName[2:-1].split(',')[2])+1])
            if v.varName[0] == 'y':
                branch_dict[int(v.varName[2:-1].split(',')[0])+1] = int(v.varName[2:-1].split(',')[1])
    print(f"target {target} computed time taken {(time.time() - start_time)/60:}")

optimizing...
target 0.01 computed time taken 0.023477296034495037
optimizing...
target 0.02 computed time taken 0.024981717268625896
optimizing...
target 0.03 computed time taken 0.03667974869410197
optimizing...
target 0.04 computed time taken 0.028430430094401042
optimizing...
target 0.05 computed time taken 0.02457118034362793
optimizing...
target 0.06 computed time taken 0.027120681603749593
optimizing...
target 0.07 computed time taken 0.030775352319081625
optimizing...
target 0.08 computed time taken 0.02061848243077596
optimizing...
target 0.09 computed time taken 0.02765376170476278
optimizing...
target 0.1 computed time taken 0.02712634801864624
optimizing...
target 0.11 computed time taken 0.03921931982040405
optimizing...
target 0.12 computed time taken 0.0237336794535319
optimizing...
target 0.13 computed time taken 0.03274633089701335
optimizing...
target 0.14 computed time taken 0.031910812854766844
optimizing...
target 0.15 computed time taken 0.02891063690185547
optimi

AttributeError: Unable to retrieve attribute 'x'

In [41]:
assignment_df = pd.DataFrame({'banker_id':assigned_bankers, 
                              'branch_id': branch,
                              'loan_id':assigned_loans, 
                              'target':targets})

In [42]:
assignment_df = pd.merge(assignment_df, banker[['banker_id', 'capacity']], how='left', on='banker_id')

In [43]:
loan2 = loan.reset_index(names='loan_id')
bank2 = bank.reset_index(names='branch_id')

In [44]:
assignment_df = pd.merge(assignment_df, loan2, how='left', on='loan_id')
assignment_df = pd.merge(assignment_df, bank2[['branch_id','location']], how='left', on='branch_id')

In [45]:
assignment_df

,banker_id,branch_id,loan_id,target,capacity,loan_amount,loan_type,location
0,6,1,1,0.01,1000,200.0,0,"330 Collins St, Melbourne, VIC, 3000"
1,3,0,2,0.01,500,400.0,0,"Westfield Fountain Gate, 352 Princes Highway, ..."
2,4,2,3,0.01,250,100.0,0,"460 Collins St, Melbourne, VIC, 3000"
3,6,1,6,0.01,1000,400.0,0,"330 Collins St, Melbourne, VIC, 3000"
4,6,1,1,0.02,1000,200.0,0,"330 Collins St, Melbourne, VIC, 3000"
...,...,...,...,...,...,...,...,...
260,3,2,14,0.34,500,300.0,0,"460 Collins St, Melbourne, VIC, 3000"
261,5,1,0,0.34,750,300.0,1,"330 Collins St, Melbourne, VIC, 3000"
262,1,2,4,0.34,750,300.0,1,"460 Collins St, Melbourne, VIC, 3000"
263,2,0,10,0.34,250,150.0,1,"Westfield Fountain Gate, 352 Princes Highway, ..."


In [49]:
cols = ['banker_id', 'branch_id', 'capacity', 'target', 'loan_amount']
grouped_df = assignment_df[cols].groupby(['banker_id', 'branch_id','capacity', 'target']).sum().reset_index()

In [50]:
grouped_df['excess_capacity'] = grouped_df['capacity'] - grouped_df['loan_amount']

In [51]:
grouped_df[grouped_df.target == 0.01]

,banker_id,branch_id,capacity,target,loan_amount,excess_capacity
30,3,0,500,0.01,400.0,100.0
76,4,2,250,0.01,100.0,150.0
98,6,1,1000,0.01,600.0,400.0


In [53]:
grouped_df.to_csv('model_1_results.csv')

In [74]:
assignment_df[assignment_df.target==0.24]

,banker_id,loan_id,target,capacity,loan_amount,loan_type
87,2,1,0.24,250,200.0,0
88,3,3,0.24,500,100.0,0
89,1,6,0.24,750,400.0,0
90,4,7,0.24,250,100.0,0
91,3,8,0.24,500,400.0,0
92,1,0,0.24,750,300.0,1


In [ ]:
grouped_df